In [1]:
import napari_toska as nts
from skimage.morphology import skeletonize
from skimage.data import binary_blobs
from skimage.measure import label
import numpy as np
import napari

In [2]:
viewer = napari.Viewer()

In [3]:
labels = label(binary_blobs(rng=0))
skeleton = skeletonize(labels)

In [4]:
Skeleton = nts.ToskaSkeleton(labels_data=labels, neighborhood='n8')
Skeleton.analyze()

Building Graph: 100%|██████████| 48/48 [00:00<00:00, 337.86it/s]


detected malformatted label: 87 changed type from 2 (branch) -> 1 (end point)
Found isolated nodes:  [24]
Could not connect isolated node:  24


In [5]:
viewer.layers.clear()
viewer.add_layer(Skeleton)

<ToskaSkeleton layer 'labels_data' at 0x2513412e460>

Napari status bar display of label properties disabled because https://github.com/napari/napari/issues/5417 and https://github.com/napari/napari/issues/4342
Selected column branch_length


In [22]:
viewer.add_labels()

<Labels layer 'Labels' at 0x2620dc05700>

In [7]:
nx.draw(Skeleton.graph, with_labels=True)

NameError: name 'nx' is not defined